make figure showing:

- Merapi true colour
- Merapi slope
- Merapi shadow overlay map
- interferogram
- pixel offset range and azimuth


In [1]:
# import external packages
import numpy as np
import pandas as pd
import numba
from numba import vectorize
import glob # for file search
import copy
import os # operating system stuff
import re # regex
import fastparquet # fast read/write for large data structures
import sklearn.preprocessing as pre # for data normalisation
from sklearn.metrics import pairwise_distances

import geopandas as gpd
import rasterio as rio
import rasterio.mask
from rasterio.plot import plotting_extent
from shapely.geometry import Polygon
from shapely.geometry.point import Point
import pyproj
from pyproj import CRS
from inpoly import inpoly2 # for fast inpolygon checks
import utm

import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
from matplotlib import cm as mpl_cm
from matplotlib import colors as mcolors 
import matplotlib.image as mplimg

from mpl_toolkits.axes_grid1 import make_axes_locatable # for colorbar scaling
from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FormatStrFormatter

import seaborn as sns
from matplotlib import rc_file_defaults
rc_file_defaults()
# sns.set(style=None, color_codes=True)

from shapely.geometry import Polygon
from shapely.geometry.point import Point
import datetime

import configparser

from cmcrameri import cm # for scientific colourmaps

###########################
# import main local package
import SPOTSAR_main as sm


In [2]:
################ Define user INPUTS #######################
######## please edit the values of this block only ########
###########################################################

# define hillshade file
HS_FILE = './test_data/DEM/TDX_Merapi_WGS84_HS.tif'

# define lon and lat files
LON_FILE = './test_data/CSK_dsc/geo2/20200910.lon'
LAT_FILE = './test_data/CSK_dsc/geo2/20200910.lat'

# define parameter text file
PARAM_FILE = './test_data/CSK_dsc/params.txt'

# DEM, SLOPE and NDVI file
DEM_FILE = './test_data/DEM/TDX_Merapi_WGS84_5m.tif'
SLOPE_FILE = './test_data/DEM/TDX_Merapi_WGS84_5m_slope.tif'
NDVI_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/sentinel2/Sentinel_NDVI_WGS84_v2.tif'
TRUE_COL_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/sentinel2/Sentinel_true_color_WGS84.tif'
LS_MAP_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/c20200910.ls_map_dem_seg.png'
IFG_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/example_ifgs/20210217_20210218_img.diff.gc.tif'

# define map region of interest
lon_lims = [110.425, 110.45]
lat_lims = [-7.555, -7.535]



# define colour range {min max} (min = -max)
vmax = 3 # range of colourscale in meters

# define file names for data, lon and lat
DIRECTORY_PATH = "./test_data/CSK_dsc/DISP_txt2/"
# define path to ccp and ccs files
DIRECTORY_PATH_CCS = "./test_data/CSK_dsc/CCS2/"

# Set the regular expression pattern to match the file names
PATTERN1 = r"^c20210217_c20210218_disp_[0-9]+_[0-9]+\.txt$"

# Set the regular expression pattern to match the ccs file names
PATTERN_CCS1 = r"^c20210217_c20210218_ccs_[0-9]+_[0-9]+$"

# open hillshade file and re-order offset and CCS files

# open hill shade file with rasterio
DEM_HS = rio.open(HS_FILE)
SHADING = DEM_HS.read(1,masked=True) # rasterio bands are indexed from 1

# extract DEM extent
DEM_EXTENT=[DEM_HS.bounds.left,DEM_HS.bounds.right,DEM_HS.bounds.bottom,DEM_HS.bounds.top]

# read ls map
ls_map = mplimg.imread(LS_MAP_FILE)

# get world borders via geopandas

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# read parameters from text file
config = configparser.ConfigParser()
config.read(PARAM_FILE)
WIDTH = int(config.get('params', 'width'))
LINES = int(config.get('params', 'lines'))
WIDTH_CCS = int(config.get('params', 'width_ccs'))
LINES_CCS = int(config.get('params', 'lines_ccs'))
R_START = int(config.get('params', 'r_start'))
A_START = int(config.get('params', 'a_start'))
R_STEP = int(config.get('params', 'r_step'))
A_STEP = int(config.get('params', 'a_step'))
HEADING = float(config.get('params', 'heading'))
MEAN_INC = float(config.get('params', 'mean_inc'))

In [3]:
# reorder file using Post_processing.reorder_files
matching_files1 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN1,0)
matching_files_ccs1 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS1,0)
print(matching_files1)
print(matching_files_ccs1)


['c20210217_c20210218_disp_58_28.txt', 'c20210217_c20210218_disp_99_48.txt', 'c20210217_c20210218_disp_140_68.txt', 'c20210217_c20210218_disp_182_88.txt', 'c20210217_c20210218_disp_224_108.txt']
['c20210217_c20210218_ccs_58_28', 'c20210217_c20210218_ccs_99_48', 'c20210217_c20210218_ccs_140_68', 'c20210217_c20210218_ccs_182_88', 'c20210217_c20210218_ccs_224_108']


In [4]:
# load data from files into class multi-kernel
example_pairs = []
for (matching_files,matching_files_ccs) in zip([matching_files1],[matching_files_ccs1]):
    datastack = sm.Post_processing.MultiKernel(DIRECTORY_PATH,
                                            matching_files,
                                            DIRECTORY_PATH_CCS,
                                            matching_files_ccs,
                                            LAT_FILE,
                                            LON_FILE,
                                            HEADING,
                                            MEAN_INC,
                                            LINES_CCS,
                                            WIDTH_CCS)
    # We need to assign some data not stored in the disp.txt files.
    datastack.get_params_from_file_name()
    datastack.get_latlon_from_file(WIDTH)
    datastack.add_lat_lon_to_data(R_START,A_START)
    datastack.crop_stack_ccs(R_STEP,A_STEP)
    # the object datastack now has several attributes associated with the whole dataset (e.g., date1, date2, heading)
    # Next we add all the offset data (disp.txt) to the stack
    stacked_data = datastack.assign_data_to_stack(R_STEP,A_STEP)
    # The attribute 'Stack' we find a list of single-kernel objects which contain the actual offset data, ccp and ccs data and the coordinates.

    # add stack to list
    example_pairs.append(datastack)



In [5]:
import fiona
from matplotlib.colors import ListedColormap
from matplotlib import cm as mpl_cm
## find nearest elevation, slope, and NDVI values


with rio.open(DEM_FILE) as src:
    DEM_heights = src.read(1)
    print('Band1 has shape', DEM_heights.shape)
    height = DEM_heights.shape[0]
    width = DEM_heights.shape[1]
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rio.transform.xy(src.transform, rows, cols)
    DEM_lons = np.array(xs)
    DEM_lats = np.array(ys)
    DEM_extent = [src.bounds.left,src.bounds.right,src.bounds.bottom,src.bounds.top]

with rio.open(SLOPE_FILE) as src:
    DEM_slope = src.read(1)




# define map region of interest
lon_lims = [np.nanmin(DEM_lons), np.nanmax(DEM_lons)]
lat_lims = [np.nanmin(DEM_lats), np.nanmax(DEM_lats)]
crop_flag=1

# create cropping polygon from ROI
if crop_flag:
    coords = ((lon_lims[0], lat_lims[0]), (lon_lims[0], lat_lims[1]), (lon_lims[1], lat_lims[1]), (lon_lims[1], lat_lims[0]), (lon_lims[0], lat_lims[0]))
    crop_poly = Polygon(coords)
    crop_poly_geojson = gpd.GeoSeries([crop_poly])
    crop_poly_geojson.to_file('./test_data/crop_ndvi_poly.shp',crs="EPSG:4326")


with fiona.open('./test_data/crop_ndvi_poly.shp', "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

# get data and read coords from first file
with rio.open(NDVI_FILE) as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
with rasterio.open("./test_Data/ndvi_cropped.tif", "w", **out_meta) as dest:
    dest.write(out_image)

with rio.open("./test_Data/ndvi_cropped.tif") as src:
    NDVI = src.read(1)
    print('Band1 has shape', NDVI.shape)
    height = NDVI.shape[0]
    width = NDVI.shape[1]
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rio.transform.xy(src.transform, rows, cols)
    NDVI_lons = np.array(xs)
    NDVI_lats = np.array(ys)
    NDVI_extent = [src.bounds.left,src.bounds.right,src.bounds.bottom,src.bounds.top]

with rio.open(TRUE_COL_FILE) as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
with rasterio.open("./test_Data/TRUE_COL_cropped.tif", "w", **out_meta) as dest:
    dest.write(out_image)

with rio.open("./test_Data/TRUE_COL_cropped.tif") as src:
    TRUE_COL_1 = src.read(1)
    TRUE_COL_2 = src.read(2)
    TRUE_COL_3 = src.read(3)
    print('Band1 has shape', TRUE_COL_1.shape)
    height = NDVI.shape[0]
    width = NDVI.shape[1]
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rio.transform.xy(src.transform, rows, cols)
    TRUE_COL_lons = np.array(xs)
    TRUE_COL_lats = np.array(ys)
    TRUE_COL_extent = [src.bounds.left,src.bounds.right,src.bounds.bottom,src.bounds.top]

TRUE_COL = np.stack((TRUE_COL_1,TRUE_COL_2,TRUE_COL_3),axis=-1)


with rio.open(IFG_FILE) as src:
    IFG_1 = src.read(1)
    IFG_2 = src.read(2)
    IFG_3 = src.read(3)
    print('Band1 has shape', IFG_1.shape)
    height = DEM_heights.shape[0]
    width = DEM_heights.shape[1]
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rio.transform.xy(src.transform, rows, cols)
    IFG_lons = np.array(xs)
    IFG_lats = np.array(ys)
    IFG_extent = [src.bounds.left,src.bounds.right,src.bounds.bottom,src.bounds.top]

IFG = np.stack((IFG_1,IFG_2,IFG_3),axis=-1)

x= mpl_cm.get_cmap('Blues_r', 135)
y= mpl_cm.get_cmap('YlGn', 135)
z = np.vstack((x(range(135)),
                       y(range(135))))
ndvi_cmap = ListedColormap(z, name='BlYlGn')

Band1 has shape (2028, 1672)
Band1 has shape (171, 141)
Band1 has shape (1017, 838)
Band1 has shape (10171, 8384)


/var/folders/m9/32v45xkn4zx6js6htb1prb2r0000gn/T/ipykernel_86118/3451707231.py:103: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  x= mpl_cm.get_cmap('Blues_r', 135)
/var/folders/m9/32v45xkn4zx6js6htb1prb2r0000gn/T/ipykernel_86118/3451707231.py:104: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  y= mpl_cm.get_cmap('YlGn', 135)


In [10]:
%matplotlib osx

from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
# make figure
grid_size = 1000
cmap=cm.vik
clims = [-3, 3]
lon_lims = [110.425, 110.46]
lat_lims = [-7.555, -7.515]
orientation = 'vertical'

obj = example_pairs[0].Stack[1]

fig = plt.figure()

gs = GridSpec(2, 4, figure=fig)
ax00 = fig.add_subplot(gs[0, 0]) # Merapi true colour with inset
ax10 = fig.add_subplot(gs[1, 0]) # Merapi NDVI

ax01 = fig.add_subplot(gs[0, 1]) # Merapi elevation
ax11 = fig.add_subplot(gs[1, 1]) # Merapi slope
ax02 = fig.add_subplot(gs[0, 2]) # Merapi shadow + overlay
ax12 = fig.add_subplot(gs[1, 2]) # Merapi ifg
ax03 = fig.add_subplot(gs[0, 3]) # range offset
ax13 = fig.add_subplot(gs[1, 3]) # azimuth offset


ax00.imshow(TRUE_COL,extent=TRUE_COL_extent)
ax00_1 = inset_axes(ax00,width='50%',height='25%',loc=2)
ax00_1.add_patch(Rectangle((90,-15),55,25,color='white'))
world.plot(ax=ax00_1,color='lightgrey',edgecolor='black')
ax00_1.scatter(110.4,-7.54,s=5,c='red')
ax00_1.set_aspect('equal','box')
ax00_1.set_xlim((90,145))
ax00_1.set_ylim((-15,10))
ax00_1.set_axis_off()

ndvi_map = ax10.imshow(NDVI,cmap=ndvi_cmap,extent=NDVI_extent,vmin=-1,vmax=1)
plt.colorbar(ndvi_map,label='NDVI [-]',orientation=orientation)

dem_map = ax01.imshow(DEM_heights,cmap='gist_earth',extent=DEM_extent,vmin=0,vmax=3000)
ax01.imshow(SHADING, cmap=cm.grayC, alpha=0.3 , extent=DEM_extent)
plt.colorbar(dem_map,label='elevation [m]',orientation=orientation)

slope_map = ax11.imshow(DEM_slope,cmap='Greys_r',extent=DEM_extent,vmin=0,vmax=45)
plt.colorbar(slope_map,label='slope [deg.]', extend='max',orientation=orientation)

ax02.imshow(ls_map,cmap='Greys',extent=DEM_extent)
ax02.scatter([],[],s=30,c='black',edgecolor='black',label='Shadow')
ax02.scatter([],[],s=30,c='white',edgecolor='black',label='Overlay')
ax02.scatter([],[],s=30,c='grey',edgecolor='black',label='Normal or\nforeshortening')
ax02.legend(loc='best')

ax12.imshow(IFG,extent=IFG_extent)


# ax03.imshow(SHADING,cmap=cm.grayC,alpha=0.5, extent=DEM_extent)
r_off_map = ax03.hexbin(obj.Lon_off.flatten(),obj.Lat_off.flatten(),C=obj.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
plt.colorbar(r_off_map,label='range offset [m]',orientation=orientation)

# ax13.imshow(SHADING,cmap=cm.grayC,alpha=0.5, extent=DEM_extent)
a_off_map = ax13.hexbin(obj.Lon_off.flatten(),obj.Lat_off.flatten(),C=obj.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
plt.colorbar(a_off_map,label='azimuth offset [m]',orientation=orientation)

print(lon_lims,lat_lims)

for (ax,text) in zip([ax00,ax10,ax01,ax11,ax02,ax12,ax03,ax13],['A','B','C','D','E','F','G','H']):
    ax.add_artist(ScaleBar(sm.plot.get_1deg_dist(),location='upper right'))
    if ax ==ax00:
        ax.text(110.44570,-7.52014,text,fontsize=20,backgroundcolor=(1,1,1,0.3))
    else:
        ax.text(110.42633,-7.52014,text,fontsize=20,backgroundcolor=(1,1,1,0.3))
    ax.set_xlim(lon_lims)
    ax.set_ylim(lat_lims)
    ax.set_axis_off()

# fig.tight_layout()

[110.425, 110.46] [-7.555, -7.515]


2023-09-21 16:06:00.545 python[86118:1940784] +[CATransaction synchronize] called within transaction
2023-09-21 16:06:07.086 python[86118:1940784] +[CATransaction synchronize] called within transaction
2023-09-21 16:06:08.220 python[86118:1940784] +[CATransaction synchronize] called within transaction


In [7]:
# plt.close('all')